# Alpha Diversity Analysis 


This code runs alpha diversity of the GCMP data using the sepp generated phylogenetic trees. The greengenes and Silva databases were used to generate the phylogentic tree.

In [5]:
#make sure plugins are running
from qiime2 import Artifact
try:
    from qiime2.plugins.diversity.methods import alpha
except:
    from qiime2.plugins.diversity.pipelines import alpha
#from qiime2.fragment_insertion.methods import sepp
from qiime2.plugins.diversity.pipelines import alpha_phylogenetic
from qiime2.plugins.diversity.visualizers import alpha_group_significance


from qiime2.plugins.feature_table.methods import rarefy
from qiime2.plugins.feature_table.visualizers import summarize

from qiime2.metadata import Metadata
from qiime2.plugins.feature_table.methods import filter_features

from os.path import abspath,exists,join
import shutil

# Loading Data
Load feature table and phylogenetic tree as qiime2 artifacts to run alpha diversity.

In [6]:
#Check that required files exist
phylo_tree = Artifact.load("../input/insertion-tree_GCMP.qza")
metadata = Metadata.load("../input/GCMP_EMP_map_r28_no_empty_samples.txt")
#these are files that will be merged (right now lets just try the greengenes files)
feature_table_gg_m2_mucus_1000 = Artifact.load("../input/feature_table_greengenes_metaxa2_mucus_1000.qza")
feature_table_gg_m2_skeleton_1000 =Artifact.load("../input/feature_table_greengenes_metaxa2_skeleton_1000.qza")
feature_table_gg_m2_tissue_1000 = Artifact.load("../input/feature_table_greengenes_metaxa2_tissue_1000.qza")


In [7]:
#need to define dictionary for the loop
feature_tables_gg_m2_1000 = {"mucus":feature_table_gg_m2_mucus_1000, "tissue":feature_table_gg_m2_tissue_1000, "skeleton":feature_table_gg_m2_skeleton_1000}

# Alpha diversity loop

Run the alpha diversity loop for each tissue compartment using the greengenes or silva phylogenetic trees.
The defined feature table and subsequent saveing file name should be changed to reflect the phylogentic tree used for the analysis.

In [12]:
#alpha diversity loop
metrics = ['faith_pd']
for compartment,table in feature_tables_gg_m2_1000.items():
    for metric in metrics:
        print(f"Caluculating alpha diversity for {compartment} using {metric}")
        alpha_results = alpha_phylogenetic(table=feature_table,metric=metric, phylogeny=phylo_tree)
        alpha_diversity = alpha_results.alpha_diversity
        
        #save the resulting feature table to disk
        output_filename = f"adiv_{compartment}_gg_m2_1000_{metric}.qza"
        output_filepath = join("../output",output_filename)
        print(f"Saving results to:{output_filepath}")
        alpha_diversity.save(output_filepath)
        
        #Calculate alpha group significance for categorial variables
        alpha_group_sig_results = alpha_group_significance(alpha_diversity=alpha_diversity,metadata=metadata)
        alpha_group_sig_visualization = alpha_group_sig_results.visualization
        output_filename = f"adiv_{compartment}_gg_m2_1000_{metric}_group_sig.qzv"
        output_filepath = join("../output",output_filename)
        print(f"Saving significance results to {output_filepath}")
        alpha_group_sig_visualization.save(output_filepath)
        

Caluculating alpha diversity for mucus using faith_pd
Saving results to:../output/adiv_mucus_gg_m2_1000_faith_pd.qza
Saving significance results to ../output/adiv_mucus_gg_m2_1000_faith_pd_group_sig.qzv
Caluculating alpha diversity for tissue using faith_pd
Saving results to:../output/adiv_tissue_gg_m2_1000_faith_pd.qza
Saving significance results to ../output/adiv_tissue_gg_m2_1000_faith_pd_group_sig.qzv
Caluculating alpha diversity for skeleton using faith_pd
Saving results to:../output/adiv_skeleton_gg_m2_1000_faith_pd.qza
Saving significance results to ../output/adiv_skeleton_gg_m2_1000_faith_pd_group_sig.qzv
